In [3]:
%load_ext autoreload
%autoreload 2

import os
from filemanager import file_ext_search as fes
from dataclasses import dataclass, field
import numpy as np
import matplotlib.pyplot as plt
from scipy import signal

from pyproj import Proj, CRS, Transformer
from sbp_tools.sbp_track_processing import process_track, save_track_to_radex, save_track, read_segypos

crs_wgs84 = CRS.from_epsg(4326)
crs_world_mercator = CRS.from_epsg(3395)
transformer = Transformer.from_crs(crs_wgs84, crs_world_mercator, always_xy=True)

data_path = r'D:\AI69_Processing\CONV_SEGY\p001_NoNavData_NavFixed'
# track
save_to = r'D:\AI69_Processing\SeismicDataAI69_p001_NoNavData_NavFixed.txt'

is_utm = False
is_arcsec = True
coord_scaler = 1
smooth = False
year_check = False
year = 2024

is_arcsec = False
is_utm = True

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [4]:
pos_files = fes.file_ext_search('.txt', data_path, recursive=False)
print(len(pos_files))

# pos_files_LF = []
# pos_files_LF = pos_files

# for file in pos_files:
#     if 'LF1' in file:
#         pos_files_LF.append(file)

pos_files_LF = pos_files  
    

segy_pos_objs = []
bad_data_dict = {}
fine_data_dict = {}

read_segypos(pos_files_LF, fine_data_dict, bad_data_dict, segy_pos_objs, year, utm_coords = is_utm, arcsecs=is_arcsec, coord_scale=coord_scaler)
process_track(segy_pos_objs, transformer, window_length=201, smooth=smooth, utm_coords=is_utm)
save_track(segy_pos_objs, save_to, smoothed=smooth)
# save_track_to_radex(segy_pos_objs)

Searching *.txt files in directory:D:\AI69_Processing\CONV_SEGY\p001_NoNavData_NavFixed
48
File AI6820241028_015617_RAW_LF_WM_rawpos is done 1 of 48
File AI6820241028_020618_RAW_LF_WM_rawpos is done 2 of 48
File AI6820241028_021441_RAW_LF_WM_rawpos is done 3 of 48
File AI6820241028_022418_RAW_LF_WM_rawpos is done 4 of 48
File AI6820241028_023419_RAW_LF_WM_rawpos is done 5 of 48
File AI6820241028_024419_RAW_LF_WM_rawpos is done 6 of 48
File AI6820241028_025419_RAW_LF_WM_rawpos is done 7 of 48
File AI6820241028_030420_RAW_LF_WM_rawpos is done 8 of 48
File AI6820241028_031420_RAW_LF_WM_rawpos is done 9 of 48
File AI6820241028_032421_RAW_LF_WM_rawpos is done 10 of 48
File AI6820241028_033421_RAW_LF_WM_rawpos is done 11 of 48
File AI6820241028_034422_RAW_LF_WM_rawpos is done 12 of 48
File AI6820241028_035422_RAW_LF_WM_rawpos is done 13 of 48
File AI6820241028_035911_RAW_LF_WM_rawpos is done 14 of 48
File AI6820241028_040912_RAW_LF_WM_rawpos is done 15 of 48
File AI6920241028_041729_CH_RAW_L

## SORT BY FILESETS

In [3]:
from sbp_tools.sbp_fileset_sort import read_sbplog, upd_segy_path, sort_files_by_fset

segy_path = r'D:\AI69_Processing\CONV_SEGY\SBP_SORTED'
segy_files = fes.file_ext_search('.sgy', segy_path, recursive=False)

qgis_log = r'D:\AI69_Processing\ai69_sbp_filetrack_wgs84.csv'
sbp_files = read_sbplog(qgis_log, 1, -3)

upd_segy_path(sbp_files, segy_files)

Searching *.sgy files in directory:D:\AI69_Processing\CONV_SEGY\TEST_NOHEAVE


In [4]:
sbp_files[0:20]

[SegyFile(name='AI6920241031_085535_RAW_LF.sgy', fileset='ai69_id022_TransitToPoly02King_Pr', path=''),
 SegyFile(name='AI6920241029_232608_RAW_LF.sgy', fileset='ai69_id005_ABRPoly01_Pr', path=''),
 SegyFile(name='AI6920241029_111209_CH_RAW_LF.sgy', fileset='ai69_id001_ABRPoly01_Pr', path=''),
 SegyFile(name='AI6920241031_092538_RAW_LF.sgy', fileset='ai69_id022_TransitToPoly02King_Pr', path=''),
 SegyFile(name='AI6920241029_093151_CH_RAW_LF.sgy', fileset='ai69_id001_ABRPoly01_Pr', path=''),
 SegyFile(name='AI6920241029_053543_CH_RAW_LF.sgy', fileset='ai69_id108_TransitToABR_Pr', path=''),
 SegyFile(name='AI6920241030_134215_BM_WC15_7kHz_500-1000_RAW_LF.sgy', fileset='ai69_id015_ABRPoly01WC01-26_St', path=''),
 SegyFile(name='AI6920241029_064109_CH_RAW_LF.sgy', fileset='ai69_id001_ABRPoly01_Pr', path=''),
 SegyFile(name='AI6920241029_054650_CH_RAW_LF.sgy', fileset='ai69_id108_TransitToABR_Pr', path=''),
 SegyFile(name='AI6920241029_141003_CH_RAW_LF.sgy', fileset='ai69_id002_ABRPoly01_Pr

In [5]:
path_sort_to = segy_path
sort_files_by_fset(sbp_files, path_sort_to, is_acf=False)

## REPLICA

In [8]:
@dataclass
class SGYFile:
    sgy_dir: str
    sgy_prof: str
    sgy_file: str
    sgy_fileproc: str
    

data_path = r'D:\AI69_Processing\CONV_SEGY\SBP_SORTED'
data_path = r'D:\AI69_Processing\CONV_SEGY\TEST_NOHEAVE'
save_replica_path = r'D:\AI69_Processing\CONV_SEGY'
replica_name = r'replica_ai69_NOHEAVE.txt'
segy_files = []


sgy_files = fes.file_ext_search('.sgy', data_path, recursive=True)

for filepath in sgy_files:
    sgy_file_name = os.path.splitext(os.path.basename(filepath))[0]
    sgy_dir = os.path.dirname(filepath)
    sgy_prof = os.path.basename(sgy_dir)
    
    if '_proc' in sgy_file_name:
        pass
    else:
        segy_files.append(SGYFile(sgy_dir=sgy_dir,
                                  sgy_prof=sgy_prof,
                                  sgy_file=sgy_file_name,
                                  sgy_fileproc=sgy_file_name+'_proc'))
        
# Replica
with open(os.path.join(save_replica_path, replica_name), 'w') as f1:
    f1.write('prof_folder\tprofile\n')
    
    prof_dir = {}
    
    for file in segy_files:
        if file.sgy_prof in prof_dir.keys():
            pass
        else:
            prof_dir[file.sgy_prof] = file.sgy_dir
        
        
    keys = sorted(list(prof_dir.keys()))

    for prof in keys:    
        f1.write(f'{prof_dir[prof]}\t{prof}\n')

Searching *.sgy files in directory:D:\AI69_Processing\CONV_SEGY\TEST_NOHEAVE
